In [1]:
from typing import Tuple

#from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
torch.manual_seed(99)

from matplotlib import pyplot as plot

In [2]:
df = pd.read_csv("./data/datasets/historical.data")
#df = df.set_index("date").sort_index(ascending=True)
df = df.sort_values(by="date",ascending=True)
df["month"] = df.apply(lambda x: int(x["date"].split("-")[1])-1, axis=1) # category indexed to 0
df["day"] = df.apply(lambda x: int(x["date"].split(" ")[0].split("-")[2])-1, axis=1) # category indexed to 0
df["hour"] = df.apply(lambda x: int(x["date"].split(" ")[1].split(":")[0]), axis=1)
df["minute"] = df.apply(lambda x: int(x["date"].split(" ")[1].split(":")[1]), axis=1)
df = df.drop(["date"], axis=1)
#df.apply(lambda x: x["date"].split(" ")[0].split("-")[1] + "-" + x["date"].split(" ")[0].split("-")[2], axis=1)
#display(df)
df_inside = df.loc[df["label"] == "inside"].drop(["label"], axis=1)
df_outside = df.loc[df["label"] == "outside"].drop(["label"], axis=1)
print("inside")
display(df_inside)
#print("outside")
#display(df_outside)

inside


,temperature,month,day,hour,minute
1,22.875,1,21,0,30
2,22.937,1,21,1,0
4,23.062,1,21,1,30
6,23.125,1,21,2,0
8,23.187,1,21,2,30
...,...,...,...,...,...
97001,24.812,10,29,21,30
97002,24.750,10,29,22,0
97005,24.500,10,29,22,30
97007,24.437,10,29,23,0


In [3]:



inside_arr=np.array(df_inside)
outside_arr=np.array(df_outside)

#print(inside_arr)

#train_test_split(inside_arr, shuffle=False)

def split_data(arr: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    total_len,_=arr.shape

    train_p = 0.80
    test_p = 0.10
    ivs_p = 0.10

    train_len = round(total_len*train_p)
    test_len = round(total_len*test_p)
    ivs_len = round(total_len*ivs_p)

    model_len = train_len+test_len

    total_used_len = model_len+ivs_len

    train = arr[:train_len]
    test = arr[train_len:model_len]
    ivs = arr[model_len:total_used_len]

    return train, test, ivs


inside_train, inside_test, inside_ivs = split_data(inside_arr)

print("train")
print(inside_train)
print("test")
print(inside_test)
print("ivs")
print(inside_ivs)

train
[[22.875  1.    21.     0.    30.   ]
 [22.937  1.    21.     1.     0.   ]
 [23.062  1.    21.     1.    30.   ]
 ...
 [23.375  4.    12.     8.     0.   ]
 [23.437  4.    12.     8.    30.   ]
 [23.375  4.    12.     9.     0.   ]]
test
[[23.375  4.    12.     9.    30.   ]
 [23.437  4.    12.    10.     0.   ]
 [23.375  4.    12.    10.    30.   ]
 ...
 [25.125  7.    21.     3.    30.   ]
 [25.062  7.    21.     4.     0.   ]
 [25.062  7.    21.     4.    30.   ]]
ivs
[[25.125  7.    21.     5.     0.   ]
 [25.062  7.    21.     5.    30.   ]
 [25.125  7.    21.     6.     0.   ]
 ...
 [24.5   10.    29.    22.    30.   ]
 [24.437 10.    29.    23.     0.   ]
 [24.562 10.    29.    23.    30.   ]]


In [4]:
# 2h30

per_min = (2*60) + 30
per_30_mins = per_min / 30

print("Number of Wanted Predictions: " + str(per_30_mins))

Number of Wanted Predictions: 5.0


One-step-ahead (auto-regressive)
<br>
One-step-ahead (auto-regressive with exogenous inputs)
<br>
Recursive
<br>
MIMO
<br>
Direct
<br>
Hybrid (Recursive and Direct)
<br>
Hybrid (Direct and MIMO)
<br>
<br>
MIMO is good too, but it would be more accurate and faster to compute several dedicated models for each step instead of all in a batch.
<br>
Direct and Hybrid(Direct and MIMO) are on top - they can execute all
<br>
<br>
Hybrid(Direct and MIMO) = [2,2,1] predictive models
<br>
Direct = [+-,1,1,1,1] predictive models
<br>
<br>
Other tests that could be implemented would be creating a Long Short-Term Memory model, trying out some kind of BERT model with months as categorial variables, or classical statistic models like SVMs or Decision Trees.

direct_0 = []
direct_0_X = []
direct_0_Y = []

direct_1 = []
direct_1_X = []
direct_1_Y = []

direct_2 = []
direct_2_X = []
direct_2_Y = []

direct_3 = []
direct_3_X = []
direct_3_Y = []

direct_4 = []
direct_4_X = []
direct_4_Y = []

direct_5 = []
direct_5_X = []
direct_5_Y = []

_temp_arr = []
_n = 0
# temp, month, day, hour
for temp, month, day, hour, minute in inside_train:
    if len(_temp_arr) < 20:
        _temp_arr.append(temp)
        _n+=1
        continue
    else:
        _temp_arr.pop(0)
        _temp_arr.append(temp)

    if _n < len(inside_train)-1:
        direct_0_X.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute])
        
        direct_0_Y.append(inside_train[_n+1][0])

        direct_0.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute,
            inside_train[_n+1][0]
            ])

    if _n < len(inside_train)-2:
        direct_1_X.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute])
        
        direct_1_Y.append(inside_train[_n+2][0])

        direct_1.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute,
            inside_train[_n+2][0]
            ])

    if _n < len(inside_train)-3:
        direct_2_X.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute])
        
        direct_2_Y.append(inside_train[_n+3][0])

        direct_2.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute,
            inside_train[_n+3][0]
            ])


    if _n < len(inside_train)-4:
        direct_3_X.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute])
        
        direct_3_Y.append(inside_train[_n+4][0])

        direct_3.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute,
            inside_train[_n+4][0]
            ])

    if _n < len(inside_train)-5:
        direct_4_X.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute])
        
        direct_4_Y.append(inside_train[_n+5][0])

        direct_4.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute,
            inside_train[_n+5][0]
            ])


    if _n < len(inside_train)-6:
        direct_5_X.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute])
        
        direct_5_Y.append(inside_train[_n+6][0])

        direct_5.append([
            _temp_arr[0], _temp_arr[1], _temp_arr[2], _temp_arr[3], _temp_arr[4], _temp_arr[5], _temp_arr[6], _temp_arr[7], _temp_arr[8], _temp_arr[9],
            _temp_arr[10], _temp_arr[11], _temp_arr[12], _temp_arr[13], _temp_arr[14], _temp_arr[15], _temp_arr[16], _temp_arr[17], _temp_arr[18], _temp_arr[19],
            month, day, hour, minute,
            inside_train[_n+6][0]
            ])

    _n+=1

direct_df = pd.DataFrame(direct_2)
direct_df.columns = ["temp0", "temp1", "temp2", "temp3", "temp4", "temp5", "temp6", "temp7", "temp8", "temp9", 
"temp10", "temp11", "temp12", "temp13", "temp14", "temp15", "temp16", "temp17", "temp18", "curr_temp", 
"month", "day", "hour", "minute",
"Y"
]
display(direct_df)

In [6]:
inside_train_0 = []
inside_train_X_0 = []
inside_train_Y_0 = []

imput_dim_df = 48 # 24H input

_temp_arr = []
_n = 0
for temp, month, day, hour, minute in inside_train:
    if len(_temp_arr) < imput_dim_df:
        _temp_arr.append(temp)
        _n+=1
        continue
    else:
        _temp_arr.pop(0)
        _temp_arr.append(temp)

    if _n < len(inside_train)-1:
        inside_train_X_0.append(torch.Tensor(_temp_arr))
        inside_train_Y_0.append(torch.Tensor([inside_train[_n+1][0]]))
        
        inside_train_0.append((torch.Tensor(_temp_arr), inside_train[_n+1][0]))

    _n+=1

#inside_train_X_0 = np.array(inside_train_X_0, dtype=np.float16)
#inside_train_Y_0 = np.array(inside_train_Y_0, dtype=np.float16)


input_dim = 48 # 24H input
hidden_dim = 1 # only 1 output
#hidden_dim = 5 # entire 2H30 output
n_layers = 1

lstm_layer = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)


batch_size = 1
seq_len = 1

inp = torch.tensor(inside_train_X_0[0])
hidden_state = torch.tensor(inside_train_Y_0[0])
cell_state = torch.tensor(inside_train_X_0[0][len(inside_train_X_0[0])-1])
hidden = (hidden_state, cell_state)

#out, hidden = lstm_layer(inside_train_X_0[0], inside_train_Y_0[0])
out, hidden = lstm_layer(inp, hidden)
print("Output shape: ", out.shape)
print("Hidden: ", hidden)

In [7]:
X, Y = inside_train_0[0]
print(X)

tensor([22.9370, 23.0620, 23.1250, 23.1870, 23.3750, 23.1250, 23.0620, 23.0000,
        22.9370, 22.9370, 22.9370, 22.8750, 22.8120, 22.6870, 22.6870, 22.6250,
        22.5620, 22.5000, 22.4370, 22.4370, 22.4370, 22.3750, 22.3750, 22.3750,
        22.3120, 22.2500, 22.2500, 22.3120, 22.3120, 22.3120, 22.2500, 22.1870,
        22.1870, 22.2500, 22.4370, 22.3750, 22.8120, 22.6870, 22.8120, 22.8120,
        22.9370, 23.1250, 23.3750, 23.3120, 23.3120, 23.3750, 23.2500, 23.3120])


In [8]:
train_loader = DataLoader(inside_train_0, batch_size=1, shuffle=False)

X, y = next(iter(train_loader))
print(X.shape)
print(X)

torch.Size([1, 48])
tensor([[22.9370, 23.0620, 23.1250, 23.1870, 23.3750, 23.1250, 23.0620, 23.0000,
         22.9370, 22.9370, 22.9370, 22.8750, 22.8120, 22.6870, 22.6870, 22.6250,
         22.5620, 22.5000, 22.4370, 22.4370, 22.4370, 22.3750, 22.3750, 22.3750,
         22.3120, 22.2500, 22.2500, 22.3120, 22.3120, 22.3120, 22.2500, 22.1870,
         22.1870, 22.2500, 22.4370, 22.3750, 22.8120, 22.6870, 22.8120, 22.8120,
         22.9370, 23.1250, 23.3750, 23.3120, 23.3120, 23.3750, 23.2500, 23.3120]])


In [9]:
class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 1

        self.lstm = nn.LSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out

learning_rate = 5e-5
num_hidden_units = 16

model = ShallowRegressionLSTM(num_sensors=1, hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")

def test_model(data_loader, model, loss_function):
    
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")


model.train()

out = model(inside_train_X_0[0])
print(out)
print(inside_train_Y_0[0])

#print("Untrained test\n--------")
#test_model(test_loader, model, loss_function)
#print()

#for ix_epoch in range(2):
#    print(f"Epoch {ix_epoch}\n---------")
#    train_model(train_loader, model, loss_function, optimizer=optimizer)
#    test_model(test_loader, model, loss_function)
#    print()

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors